# <font color='#330066'>Data Wrangling</font>

In this notebook the first step of the project - Data Wrangling - is shown.
<img src="https://github.com/asyaparfenova/air_condidioner_effectiveness_cost_carbon_footprint/blob/main/images/data_wrangling.png?raw=true" alt="Data Wrangling Goal" style="width: 600px;"/>

The row data had the form of one table with over two million rows, each of them contained sensor type and name, various timestamps, values (numerical for measurements like temperature and categorical for event-sensors) and some other attributes.

The main challenge of data wrangling was to pivot this table into time-series or, more specifically, sequence of successive equally spaced points in time with values for each sensor.

In [13]:
import pandas as pd
import numpy as np
from scipy import stats
import warnings

In [14]:
warnings.simplefilter(action='ignore', category=FutureWarning)

### <font color='#330066'>Step 1</font>

First, let's have a look at our data.

In [72]:
data = pd.read_csv('data/full_data.csv', sep=',', index_col='Unnamed: 0')

In [42]:
data.head()

,state_id,entity_id,state,last_changed,last_updated
0,1406062,sensor.room_temperature,21.7,2020-05-01 00:01:22,2020-05-01 00:01:22
1,1406063,sensor.room_humidity,52.6,2020-05-01 00:01:22,2020-05-01 00:01:22
2,1406064,sensor.bathroom_temperature,22.7,2020-05-01 00:01:22,2020-05-01 00:01:22
3,1406065,sensor.bathroom_humidity,56.0,2020-05-01 00:01:22,2020-05-01 00:01:22
4,1406066,sensor.fittonia_temperature,21.1,2020-05-01 00:01:22,2020-05-01 00:01:22


In [43]:
data.shape

(2412216, 5)

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2412216 entries, 0 to 2412215
Data columns (total 5 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   state_id      int64 
 1   entity_id     object
 2   state         object
 3   last_changed  object
 4   last_updated  object
dtypes: int64(1), object(4)
memory usage: 110.4+ MB


In [45]:
data.isna().sum()

state_id           0
entity_id          0
state           3012
last_changed       0
last_updated       0
dtype: int64

**Which and how many different sensors do we have?**

In [46]:
data['entity_id'].unique()

array(['sensor.room_temperature', 'sensor.room_humidity',
       'sensor.bathroom_temperature', 'sensor.bathroom_humidity',
       'sensor.fittonia_temperature', 'sensor.fittonia_moisture',
       'sensor.fittonia_fertility', 'sensor.fittonia_lux',
       'sensor.kitchen_temperature', 'sensor.kitchen_humidity',
       'plant.fittonia', 'sensor.kitchen_thermostat_temperature',
       'climate.kitchen_thermostat', 'binary_sensor.kitchen_window',
       'sensor.equipment_temperature', 'binary_sensor.entrance_door',
       'person.one', 'sensor.weather_temperature', 'light.balcony',
       'sensor.weather_humidity', 'light.kitchen_rgb',
       'sensor.bathroom_thermostat_temperature',
       'climate.bathroom_thermostat',
       'sensor.room_thermostat_temperature', 'climate.room_thermostat',
       'light.hallway_rgb', 'binary_sensor.critical', 'light.bathroom',
       'switch.raspberry_pi_fan', 'binary_sensor.balcony_door',
       'light.bedside_lamp', 'light.hue_go', 'media_player.tv',


In [47]:
data['entity_id'].unique().shape[0]

77

### <font color='#330066'>Step 2</font>

Let's drop the information, that we obviously don't need, such as information about plants, light (all light sources are LED, therefore they do not affect the temperature), devices, other than our air conditioner unit and so on.

In [48]:
sensors_to_drop = ['sensor.calathea_temperature',
                   'plant.calathea',
                   'sensor.fittonia_temperature',
                   'plant.fittonia',
                   'sensor.room_illuminance_lux',
                   'sensor.room_illuminance_lux_2',
                   'sensor.calathea_lux',
                   'sensor.calathea_sunlight_accumulated',
                   'sensor.calathea_fertility',
                   'sensor.fittonia_fertility',
                   'sensor.fittonia_lux',
                   'sensor.fittonia_moisture',
                   'sensor.fittonia_sunlight_accumulated',
                   'binary_sensor.warning',
                   'switch.raspberry_pi_fan',
                   'media_player.tv',
                   'binary_sensor.laptop_work',
                   'binary_sensor.critical',
                   'binary_sensor.laptop',
                   'binary_sensor.printer',
                   'vacuum.roborock',
                   'switch.phone_charger',
                   'sensor.calathea_moisture',
                   'light.office_ceiling',
                   'light.kitchen_rgb',
                   'light.balcony',
                   'light.hue_go',
                   'light.kitchen_ceiling',
                   'light.room_ceiling',
                   'light.bathroom',
                   'light.bedside_lamp',
                   'light.desk_lamp',
                   'light.hallway_rgb',
                   'light.hallway_ceiling',
                   'light.blaulicht',
                   'binary_sensor.entrance_door',
                   'light.room_corner',
                   'light.storage',
                   'sensor.washing_machine_vibration',
                   'sensor.washing_machine_vibration_strength',
                   'sensor.kitchen_thermostat_temperature',
                   'climate.kitchen_thermostat',
                   'sensor.bathroom_thermostat_temperature',
                   'climate.bathroom_thermostat',
                   'sensor.room_thermostat_temperature',
                   'climate.room_thermostat',
                   'sensor.kitchen_temperature_3',
                   'sensor.kitchen_temperature_2',
                   'sensor.kitchen_illuminance_lux',
                   'sensor.hallway_temperature',
                   'sensor.sunlight_lux',
                   'sensor.sysmon_cpu_use',
                   'sensor.sysmon_cpu_temperature',
                   'fan.fan',
                   'sensor.bathroom_temperature_2',
                   'sensor.room_temperature_2']

In [73]:
for item in sensors_to_drop:
    data.drop(data[data['entity_id'] == item].index, axis=0, inplace=True)

In [74]:
data.shape

(928315, 5)

The "state_id" column also doesn't carry any important for our project information

In [75]:
data.drop(['state_id'], axis=1, inplace=True)

In [76]:
data.shape

(928315, 4)

### <font color='#330066'>Step 3</font>

Dropping NaNs and duplicates

In [77]:
data.dropna(inplace=True)
data.shape

(928287, 4)

In [78]:
data.duplicated().sum()

21075

In [79]:
data.drop_duplicates(keep=False,inplace=True)
data.shape

(891427, 4)

### <font color='#330066'>Step 4</font>

If we will try to pivot the table at this stage,we'll get the error message:

<font color='red'>**ValueError:**</font> Index contains duplicate entries, cannot reshape

The problem is in our data quality, despite we got rid of NaNs and duplicates, sometimes we have several recordings with different values from the same sensor and the same timestamp.

Let's see some examples.

In [80]:
grouped_data = data[['entity_id', 'state', 'last_updated']].groupby(['entity_id', 'last_updated']).count()

In [81]:
grouped_data['state'].unique()

array([1, 2, 3], dtype=int64)

In [82]:
grouped_data[grouped_data['state'] == 3]

,,state
entity_id,last_updated,
cover.balcony,2020-09-19 11:36:41,3
cover.windows,2020-09-19 11:36:41,3
sensor.equipment_temperature,2020-05-24 07:05:19,3


In [83]:
data[data['entity_id'] == 'sensor.equipment_temperature'][265:270]

,entity_id,state,last_changed,last_updated
26213,sensor.equipment_temperature,30.1,2020-05-03 10:12:40,2020-05-03 10:44:45
26336,sensor.equipment_temperature,30.6,2020-05-03 10:44:45,2020-05-03 11:00:17
26337,sensor.equipment_temperature,30.3,2020-05-03 11:00:17,2020-05-03 11:00:17
26464,sensor.equipment_temperature,30.3,2020-05-03 11:00:17,2020-05-03 11:18:31
26660,sensor.equipment_temperature,30.5,2020-05-03 11:18:31,2020-05-03 11:47:25


We'll drop artefacts by creating a colunm 'updated_shift' by shifting the timestamp by one row and then simply deleting the rows, where 'last_updated' and 'updated_shift' timestamps are equal.

In [84]:
data['updated_shift'] = data['last_updated'].shift()
data.drop(data[data['last_changed'] == data['updated_shift']].index, axis=0, inplace = True)
data.shape

(480662, 5)

In [85]:
grouped_data = data[['entity_id', 'state', 'last_updated']].groupby(['entity_id', 'last_updated']).count()
grouped_data['state'].unique()

array([1], dtype=int64)

Now we got rid of "timestamp-duplicates".

Another artefact: some binary sensor values ('unavailable', 'unknown', '\\N', 'None', 'ok') are not meaningful for our research and modelling. Let's drop them too.

In [70]:
data['state'].unique()

array(['off', '26.6', '26.5', 'not_home', '26.4', '26.7', 'home', '26.8',
       'open', '27.0', '27.2', 'unknown', '27.3', '27.5', '27.4', '27.6',
       '27.7', '27.9', '27.8', '28.0', '27.1', '26.9', '26.3', '26.2',
       '28.4', '28.9', '29.4', '29.5', '29.9', '30.5', '31.0', '30.8',
       '31.4', '31.3', '31.5', '32.0', '31.8', '31.7', '32.3', '32.1',
       '32.2', '31.2', '30.9', '30.4', '29.7', '29.2', '29.0', '28.7',
       '28.2', '28.1', '28.6', '28.5', '29.6', '29.3', '30.1', '30.6',
       '30.3', '31.6', '32.7', '32.4', '30.7', '30.2', '31.1', '28.8',
       '28.3', '26.1', '25.9', '25.8', '30.0', 'on', '29.8', '31.9',
       '32.6', '32.5', '33.0', '33.1', '33.2', '33.5', '33.7', '34.2',
       '34.3', '34.7', '34.8', '35.0', '34.9', '35.2', '34.4', '33.9',
       '33.3', '32.8', '29.1', 'cool', '26.0', '25.6', '25.7', '25.5',
       '25.3', '25.0', '24.5', '24.3', '24.0', '23.7', '23.6', '23.5',
       '23.3', '23.2', '24.1', '24.4', '24.7', '24.9', '25.1', '25.4',
  

In [86]:
data.drop(data[data['state'] == 'unavailable'].index, axis=0, inplace=True)
data.drop(data[data['state'] == 'unknown'].index, axis=0, inplace=True)
data.drop(data[data['state'] == '\\N'].index, axis=0, inplace=True)
data.drop(data[data['state'] == 'None'].index, axis=0, inplace=True)
data.drop(data[data['state'] == 'ok'].index, axis=0, inplace=True)

data.shape

(477838, 5)

Perfect! No more dupicates! Now we can pivot!

### <font color='#330066'>Step 5</font>

Dropping excessive columns and converting timestamp into *datetime* format

In [87]:
data.drop(['last_changed', 'updated_shift'], axis=1, inplace=True)
data.shape

(477838, 3)

In [88]:
data['timestamp'] = pd.to_datetime(data['last_updated'])
data.drop(['last_updated'], axis=1, inplace=True)
data.shape

(477838, 3)

### <font color='#330066'>Step 6</font>

Not table-pivot is possible

In [89]:
df = data.pivot(index='timestamp', columns='entity_id', values='state')

In [90]:
df.shape

(443739, 21)

In [91]:
df.head(3)

entity_id,binary_sensor.balcony_door,binary_sensor.kitchen_window,binary_sensor.room_window,climate.air_conditioner,climate.air_conditioner_old,cover.balcony,cover.windows,person.one,person.two,sensor.ac_power,...,sensor.bathroom_humidity,sensor.bathroom_temperature,sensor.equipment_temperature,sensor.kitchen_humidity,sensor.kitchen_temperature,sensor.office_co2,sensor.room_humidity,sensor.room_temperature,sensor.weather_humidity,sensor.weather_temperature
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-05-01 00:01:22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.7,NaN,NaN
2020-05-01 00:03:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.7,NaN,NaN
2020-05-01 00:03:41,NaN,off,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### <font color='#330066'>Step 7</font>

For futher manipulations like grouping data by time it is handy to split the data to numerical and categorical, since procedures will be different.

In [92]:
numeric_columns = ['sensor.ac_power',
       'sensor.atmospheric_pressure', 'sensor.bathroom_humidity',
       'sensor.bathroom_temperature',
       'sensor.equipment_temperature', 'sensor.kitchen_humidity',
       'sensor.kitchen_temperature',
       'sensor.office_co2', 'sensor.room_humidity', 'sensor.room_temperature', 'sensor.weather_humidity',
       'sensor.weather_temperature']

In [93]:
categorical_columns = ['binary_sensor.balcony_door', 'binary_sensor.kitchen_window',
       'binary_sensor.room_window', 'climate.air_conditioner', 'climate.air_conditioner_old', 'cover.balcony',
       'cover.windows', 'person.one', 'person.two']

### <font color='#330066'>Step 8</font>

**Working with numeric data**

First, lets convert "numeric" data to actually numeric format

In [94]:
for column in numeric_columns:
    df[column] = pd.to_numeric(df[column]) 

Then we use a bit tricky procedure: we split the timestamp on month, day, hour and minute values. It will allow us to group data by every minute and create a properly structured time-series with successive equally spaced (by 1 minute) points in time

In [95]:
numeric_df = df[numeric_columns].groupby([df.index.month, df.index.day, df.index.hour, df.index.minute]).mean()
numeric_df.shape

(179068, 12)

In [96]:
numeric_df['check_time'] = numeric_df.index
numeric_df['check_time_2'] = numeric_df['check_time'].transform(lambda x: f'2020-{x[0]}-{x[1]} {x[2]}:{x[3]}:00')
numeric_df['time'] = pd.to_datetime(numeric_df['check_time_2'])

del numeric_df['check_time']
del numeric_df['check_time_2']

numeric_df.shape

(179068, 13)

As a result we have a time column rounded to one minute

In [101]:
print (numeric_df['time'].head(3),'\n\n',numeric_df['time'].tail(3))

timestamp  timestamp  timestamp  timestamp
5          1          0          1           2020-05-01 00:01:00
                                 3           2020-05-01 00:03:00
                                 5           2020-05-01 00:05:00
Name: time, dtype: datetime64[ns] 

 timestamp  timestamp  timestamp  timestamp
9          30         23         57          2020-09-30 23:57:00
                                 58          2020-09-30 23:58:00
                                 59          2020-09-30 23:59:00
Name: time, dtype: datetime64[ns]


### <font color='#330066'>Step 9</font>

**Working with categorical data**

Let's repeat the steps with one difference, for grouping categorical data we can't use mean() function, therefore we call on mode() function (we used *stats* module of *scipy* library)

In [102]:
categorical_df = df[categorical_columns].groupby([df.index.month,
                                                  df.index.day,
                                                  df.index.hour,
                                                  df.index.minute]).agg(lambda x: stats.mode(x)[0][0])
categorical_df.shape

(179068, 9)

In [103]:
categorical_df['check_time'] = categorical_df.index
categorical_df['check_time_2'] = categorical_df['check_time'].transform(lambda x: f'2020-{x[0]}-{x[1]} {x[2]}:{x[3]}:00')
categorical_df['time'] = pd.to_datetime(categorical_df['check_time_2'])
del categorical_df['check_time']
del categorical_df['check_time_2']

categorical_df.shape

(179068, 10)

In [104]:
print (categorical_df['time'].head(3),'\n\n',categorical_df['time'].tail(3))

timestamp  timestamp  timestamp  timestamp
5          1          0          1           2020-05-01 00:01:00
                                 3           2020-05-01 00:03:00
                                 5           2020-05-01 00:05:00
Name: time, dtype: datetime64[ns] 

 timestamp  timestamp  timestamp  timestamp
9          30         23         57          2020-09-30 23:57:00
                                 58          2020-09-30 23:58:00
                                 59          2020-09-30 23:59:00
Name: time, dtype: datetime64[ns]


### <font color='#330066'>Step 10</font>

Creating continuous time dataframe 

In [109]:
times = pd.date_range('2020-05-01', '2020-10-01', freq='1min')

times[:20]

DatetimeIndex(['2020-05-01 00:00:00', '2020-05-01 00:01:00',
               '2020-05-01 00:02:00', '2020-05-01 00:03:00',
               '2020-05-01 00:04:00', '2020-05-01 00:05:00',
               '2020-05-01 00:06:00', '2020-05-01 00:07:00',
               '2020-05-01 00:08:00', '2020-05-01 00:09:00',
               '2020-05-01 00:10:00', '2020-05-01 00:11:00',
               '2020-05-01 00:12:00', '2020-05-01 00:13:00',
               '2020-05-01 00:14:00', '2020-05-01 00:15:00',
               '2020-05-01 00:16:00', '2020-05-01 00:17:00',
               '2020-05-01 00:18:00', '2020-05-01 00:19:00'],
              dtype='datetime64[ns]', freq='T')

In [111]:
dataframe = pd.DataFrame(index=times)
dataframe.shape

(220321, 0)

Joining our data to continuous time series

In [112]:
dataframe = dataframe.join(numeric_df.set_index('time')) #joining numeric data
dataframe = dataframe.join(categorical_df.set_index('time')) #joining categorical data
dataframe.shape

(220321, 21)

Filling NaN values of our sparse dataframe, created as a result of left join

In [115]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 220321 entries, 2020-05-01 00:00:00 to 2020-10-01 00:00:00
Freq: T
Data columns (total 21 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   sensor.ac_power               92133 non-null   float64
 1   sensor.atmospheric_pressure   3585 non-null    float64
 2   sensor.bathroom_humidity      690 non-null     float64
 3   sensor.bathroom_temperature   18489 non-null   float64
 4   sensor.equipment_temperature  9973 non-null    float64
 5   sensor.kitchen_humidity       736 non-null     float64
 6   sensor.kitchen_temperature    18058 non-null   float64
 7   sensor.office_co2             81117 non-null   float64
 8   sensor.room_humidity          635 non-null     float64
 9   sensor.room_temperature       50882 non-null   float64
 10  sensor.weather_humidity       449 non-null     float64
 11  sensor.weather_temperature    15455 non-null   float64
 12  bi

In [116]:
dataframe.sample(5)

,sensor.ac_power,sensor.atmospheric_pressure,sensor.bathroom_humidity,sensor.bathroom_temperature,sensor.equipment_temperature,sensor.kitchen_humidity,sensor.kitchen_temperature,sensor.office_co2,sensor.room_humidity,sensor.room_temperature,...,sensor.weather_temperature,binary_sensor.balcony_door,binary_sensor.kitchen_window,binary_sensor.room_window,climate.air_conditioner,climate.air_conditioner_old,cover.balcony,cover.windows,person.one,person.two
2020-07-11 05:01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,466.0,NaN,NaN,...,NaN,0,0,0,0,0,0,0,0,0
2020-08-08 10:36:00,1068.25,NaN,NaN,NaN,NaN,NaN,NaN,406.0,NaN,NaN,...,NaN,0,0,0,0,0,0,0,0,0
2020-09-30 23:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-11 05:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-10 06:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


First, let's reverse "0" values a actually created from 'NaN' values while using groupby method

In [117]:
dataframe.replace(0, np.NaN, inplace=True)

In [118]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 220321 entries, 2020-05-01 00:00:00 to 2020-10-01 00:00:00
Freq: T
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   sensor.ac_power               91911 non-null  float64
 1   sensor.atmospheric_pressure   3585 non-null   float64
 2   sensor.bathroom_humidity      690 non-null    float64
 3   sensor.bathroom_temperature   18489 non-null  float64
 4   sensor.equipment_temperature  9973 non-null   float64
 5   sensor.kitchen_humidity       736 non-null    float64
 6   sensor.kitchen_temperature    18058 non-null  float64
 7   sensor.office_co2             81117 non-null  float64
 8   sensor.room_humidity          635 non-null    float64
 9   sensor.room_temperature       50882 non-null  float64
 10  sensor.weather_humidity       449 non-null    float64
 11  sensor.weather_temperature    15455 non-null  float64
 12  binary_sensor.ba

In [119]:
dataframe.fillna(method='ffill', inplace = True)

dataframe.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 220321 entries, 2020-05-01 00:00:00 to 2020-10-01 00:00:00
Freq: T
Data columns (total 21 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   sensor.ac_power               187995 non-null  float64
 1   sensor.atmospheric_pressure   135133 non-null  float64
 2   sensor.bathroom_humidity      217174 non-null  float64
 3   sensor.bathroom_temperature   220310 non-null  float64
 4   sensor.equipment_temperature  220315 non-null  float64
 5   sensor.kitchen_humidity       177707 non-null  float64
 6   sensor.kitchen_temperature    220278 non-null  float64
 7   sensor.office_co2             131289 non-null  float64
 8   sensor.room_humidity          177310 non-null  float64
 9   sensor.room_temperature       220320 non-null  float64
 10  sensor.weather_humidity       219735 non-null  float64
 11  sensor.weather_temperature    220302 non-null  float64
 12  bi

We still have quite a few NaN values - they are natural to our data and caused by hcanging sensors' names and introducing new sensors during our observation period. We'll leave them fot the moment.

As we can see, in the "middle" of our observation time data is clean and has desired format.

In [120]:
dataframe[132475:132480]

,sensor.ac_power,sensor.atmospheric_pressure,sensor.bathroom_humidity,sensor.bathroom_temperature,sensor.equipment_temperature,sensor.kitchen_humidity,sensor.kitchen_temperature,sensor.office_co2,sensor.room_humidity,sensor.room_temperature,...,sensor.weather_temperature,binary_sensor.balcony_door,binary_sensor.kitchen_window,binary_sensor.room_window,climate.air_conditioner,climate.air_conditioner_old,cover.balcony,cover.windows,person.one,person.two
2020-07-31 23:55:00,67.0,1015.0,43.0,23.9,29.1,50.3,24.4,400.0,41.9,23.7,...,17.1,on,off,off,off,NaN,open,open,not_home,not_home
2020-07-31 23:56:00,67.0,1015.0,43.0,23.9,29.1,50.3,24.4,400.0,41.9,23.7,...,17.0,on,off,off,off,NaN,open,open,not_home,not_home
2020-07-31 23:57:00,67.0,1015.0,43.0,23.9,29.3,50.3,24.4,400.0,41.9,23.7,...,17.0,on,off,off,off,NaN,open,open,not_home,not_home
2020-07-31 23:58:00,67.0,1015.0,43.0,23.9,29.3,50.3,24.4,400.0,41.9,23.7,...,17.0,on,off,off,off,NaN,open,open,not_home,not_home
2020-07-31 23:59:00,67.0,1015.0,43.0,23.9,29.3,50.3,24.4,400.0,41.9,23.7,...,17.0,on,off,off,off,NaN,open,open,not_home,not_home


We'll saved the transformed clean data for further EDA and ML

In [121]:
dataframe.to_csv('data/data.csv')
dataframe[87840:132480].to_csv('data/july.csv')